# TEEN PSYCHOLOGY Dataset

## 📘 Introduction

Teen mental health is a growing concern in an increasingly digital and high-pressure world. The "Inside Teen Minds" dataset offers a detailed simulation of daily mood, stress levels, habits, and digital behavior among high school students across more than 40 countries in 2025. This project uses clustering techniques to explore patterns in the mental health and lifestyle behaviors of teens. The goal is to identify distinct behavioral profiles and gain insights into how daily habits and technology use relate to emotional well-being.

---

## 📊 Dataset Description

The dataset consists of several CSV files, with the main file containing daily time-series data for 30,000 students. It includes over 35 features related to:

- Mood and stress levels  
- Sleep patterns and screen time  
- Exercise, journaling, and social interaction  
- Study productivity and AI tool usage (e.g., ChatGPT, Gemini, Notion AI)  
- Demographic information such as age, gender, and country  

Supporting files provide aggregated statistics by country, gender, and AI tool usage frequency. The dataset is well-suited for unsupervised learning, behavioral analysis, and mental health research.

The files include:

* wellness_habits_distribution.csv
* average_mood_stress_by_gender.csv
* ai_usage_by_country.csv
* ai_tool_popularity.csv
* screen_vs_sleep_by_age.csv
* daily_mood_stress_trends.csv
* average_support_feeling_by_country.csv
* modern_teen_mental_health_main.csv

The dataset can be found here: https://www.kaggle.com/datasets/dakshbhatnagar08/inside-teen-minds-global-mental-health-and-habits

---

## 🎯 Objective

The objective of this project is to apply unsupervised learning methods to discover meaningful clusters among students based on their daily habits and mental health indicators. The specific goals are:

- Conduct exploratory data analysis to understand key trends and variable distributions  
- Select relevant features for clustering based on behavioral and psychological indicators  
- Apply clustering algorithms (e.g., K-Means, Hierarchical Clustering) to group similar students  
- Analyze and label the resulting clusters to interpret behavioral patterns   

The aim is to identify common lifestyle patterns and potential risk groups, providing a clearer understanding of the diverse ways teens cope with stress, manage technology, and maintain mental well-being.

---


 #### **Note**:This is the second version of the project. Hierarchical clustering proved impractical in an earlier version. Therefore, this notebook skips EDA and focuses on clustering implementation and analysis.

In [28]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dakshbhatnagar08/inside-teen-minds-global-mental-health-and-habits")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/inside-teen-minds-global-mental-health-and-habits


In [29]:
import os
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.feature_selection import mutual_info_classif

import matplotlib.pyplot as plt

import hdbscan

from sklearn.decomposition import FactorAnalysis
from sklearn.mixture import GaussianMixture
from sklearn.cluster import SpectralClustering, AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_distances

import umap.umap_ as umap
from umap import UMAP
from collections import defaultdict

##  Main Dataset: Modern Teen Mental Health – Summary

-  **Size**: 30,000 rows × 17 columns
-  No missing values except `ai_tool`
-  **Key variables**:
  - Mental state: `mood`, `stress_level`, `support_feeling`
  - Lifestyle: `sleep_hours`, `screen_time_hours`, wellness habits
  - Social: `social_interaction_rating`, `used_ai_today`, `ai_tool`
-  Notable ranges:
  - `screen_time_hours`: -0.8 to 15.6 → check for outliers
  - `sleep_hours`: as low as 0.2 hours
-  Mood average: ~6.0 (scale 1–9)
-   Wellness habits are binary and track daily activity
-  High potential for clustering by behavior or mental health patterns

In [30]:
df = pd.read_csv(os.path.join(path, "modern_teen_mental_health_main.csv"))

In [31]:
df = df.drop(columns=['ai_tool', "country", "grade"])
# Identify categorical columns (excluding 'student_id' and 'date')
exclude = ['student_id', 'date']
categorical_cols = df.select_dtypes(include=['object', 'bool']).columns.difference(exclude)

# Perform one-hot encoding
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

'ai_tool', "country" and "grade" are dropped because they reduce the scope of the clustering.

In [32]:
df

,student_id,date,age,mood,stress_level,sleep_hours,screen_time_hours,social_interaction_rating,support_feeling,exercised_today_True,gender_Male,gender_Other,journaled_today_True,meditated_today_True,used_ai_today_True
0,S0001,2025-01-01,15,5,5,3.6,2.1,6,7,True,False,False,False,False,False
1,S0001,2025-01-02,15,6,4,9.1,8.8,4,7,True,False,False,True,False,True
2,S0001,2025-01-03,15,8,2,8.5,7.8,6,5,False,False,False,False,True,True
3,S0001,2025-01-04,15,5,6,4.9,6.1,5,9,False,False,False,True,True,False
4,S0001,2025-01-05,15,9,1,7.7,7.5,5,9,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,S1000,2025-01-26,14,3,7,8.9,6.3,5,6,True,False,True,False,True,True
29996,S1000,2025-01-27,14,5,6,5.2,9.0,3,8,True,False,True,True,False,False
29997,S1000,2025-01-28,14,3,7,6.4,9.2,8,9,False,False,True,True,True,True
29998,S1000,2025-01-29,14,5,5,7.4,4.7,3,9,False,False,True,False,False,False


## Following Steps:

Since teenage school life is largely shaped by regular school schedules, it is reasonable to assume that students’ psychological and behavioral patterns follow a weekly cycle. In other words, under typical conditions, the experience and mood of each day of the week tend to be consistent—for example, every Monday within a given month should feel broadly similar.

To test this assumption, we transform the dataset using a pivot operation that aggregates the data by day of the week for each student, enabling us to analyze these recurring weekly patterns more effectively.

In [33]:
# Step 1: Prepare data
df['date'] = pd.to_datetime(df['date'])
df['day_name'] = df['date'].dt.day_name()

# Step 2: Extract static columns
static_cols = df.groupby('student_id')[
    ['age'] + [col for col in df.columns if col.startswith(('gender_', 'country_', 'grade_'))]
].first().reset_index()

# Step 3: Drop static columns before aggregation
df_no_static = df.drop(columns=['age',
        'exercised_today_True', 'gender_Male',
       'gender_Other'])

# Step 4: Identify column types
id_cols = ['student_id', 'day_name']
numeric_cols = df_no_static.select_dtypes(include='number').columns.tolist()
object_cols = df_no_static.select_dtypes(include=['object', 'bool']).drop(columns=id_cols, errors='ignore').columns.tolist()

# Step 5: Custom aggregation
def custom_agg(x):
    if x.dtype == 'O' or x.dtype == 'bool':
        return x.mode().iloc[0] if not x.mode().empty else np.nan
    else:
        return x.mean()

grouped = df_no_static.groupby(id_cols).agg(custom_agg).reset_index()

# Step 6: Pivot and reset index
pivot_numeric = grouped.pivot(index='student_id', columns='day_name', values=numeric_cols).reset_index()
pivot_object = grouped.pivot(index='student_id', columns='day_name', values=object_cols).reset_index()

# Step 7: Flatten MultiIndex columns
pivot_numeric.columns = ['student_id'] + [f"{col}_{day.lower()}" for col, day in pivot_numeric.columns[1:]]
pivot_object.columns = ['student_id'] + [f"{col}_{day.lower()}" for col, day in pivot_object.columns[1:]]

# Step 8: Combine
final_df = static_cols.merge(pivot_numeric, on='student_id').merge(pivot_object, on='student_id')

In [34]:
final_df

,student_id,age,gender_Male,gender_Other,mood_friday,mood_monday,mood_saturday,mood_sunday,mood_thursday,mood_tuesday,...,meditated_today_True_thursday,meditated_today_True_tuesday,meditated_today_True_wednesday,used_ai_today_True_friday,used_ai_today_True_monday,used_ai_today_True_saturday,used_ai_today_True_sunday,used_ai_today_True_thursday,used_ai_today_True_tuesday,used_ai_today_True_wednesday
0,S0001,15,False,False,7.75,5.00,6.75,7.75,4.4,5.00,...,False,False,False,False,False,True,True,False,False,False
1,S0002,17,False,True,6.50,7.75,5.00,7.25,6.6,4.50,...,False,False,False,True,True,False,True,False,True,False
2,S0003,17,False,False,5.75,4.50,6.50,5.75,5.0,7.00,...,False,False,False,False,False,True,False,True,True,True
3,S0004,17,False,False,6.25,4.25,4.75,6.00,6.8,5.00,...,False,False,False,True,False,True,False,True,True,False
4,S0005,16,True,False,4.50,4.75,6.25,6.25,6.8,6.75,...,False,False,False,False,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,S0996,14,False,True,6.75,5.75,6.50,6.00,6.2,5.00,...,False,False,False,True,False,True,False,True,False,True
996,S0997,14,False,False,5.75,5.50,8.25,6.25,5.8,6.50,...,False,False,False,True,False,False,False,False,True,True
997,S0998,16,True,False,4.50,6.50,7.00,4.25,6.4,6.75,...,False,True,False,False,True,False,True,True,True,False
998,S0999,17,False,True,4.00,5.50,6.25,6.00,6.8,5.75,...,False,False,False,False,True,True,False,True,False,True


# Best Method Check

UMAP is performed with n_neighbours=20 and n_components=35.

Then, Gaussian Mixture, Spectral Clustering, Agglomerative Clustering and DBSCAN are performed for a cluster range of 2 to 12. For each one, Silhouette Score is computed.



In [35]:
# 1. Prepare data (exclude student_id)
X = final_df.drop(columns=['student_id'])

# 2. Identify numeric features (everything is numeric or dummies already)
numeric_features = X.columns.tolist()

# 3. Build preprocessor: scale all numeric features, leave rest (none) untouched
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features)
], remainder='passthrough')

# 4. UMAP reducer
umap_reducer = umap.UMAP(
    n_components=20,
    metric='cosine',
    random_state=42,
    n_neighbors=20,
    min_dist=0.0
)

# 5. Full pipeline: scaling → UMAP
pipeline = Pipeline([
    ('prep', preprocessor),
    ('umap', umap_reducer)
])

X_reduced = pipeline.fit_transform(X)
print(f"Reduced to {X_reduced.shape[1]} UMAP dimensions")

# 6. Clustering evaluation function
def evaluate(X, labels):
    unique = np.unique(labels)
    if len(unique) > 1 and not (len(unique) == 2 and -1 in unique):
        return silhouette_score(X, labels, metric='cosine')
    return np.nan

results = {}

# 7. GMM  k=2..12
for k in range(2, 13):
    gmm = GaussianMixture(n_components=k, covariance_type='full', random_state=42)
    lbl = gmm.fit_predict(X_reduced)
    results[f'GMM_{k}'] = evaluate(X_reduced, lbl)

# 8. Spectral k=2..12
for k in range(2, 13):
    spec = SpectralClustering(n_clusters=k, affinity='nearest_neighbors', random_state=42)
    lbl = spec.fit_predict(X_reduced)
    results[f'Spectral_{k}'] = evaluate(X_reduced, lbl)

# 9. Agglomerative k=2..12
for k in range(2, 13):
    agg = AgglomerativeClustering(n_clusters=k, metric='cosine', linkage='average')
    lbl = agg.fit_predict(X_reduced)
    results[f'Agglomerative_{k}'] = evaluate(X_reduced, lbl)

# 10. DBSCAN
db = DBSCAN(eps=0.5, min_samples=5, metric='cosine')
lbl = db.fit_predict(X_reduced)
results['DBSCAN'] = evaluate(X_reduced, lbl)

# 11. Report scores
for method, score in sorted(results.items(), key=lambda x: (np.nan_to_num(x[1], nan=-1)), reverse=True):
    print(f"{method}: Silhouette = {score:.3f}")

best = max(results, key=results.get)
print(f"\nBest: {best} (silhouette={results[best]:.3f})")

# 12. Fit best & assign
if best.startswith('GMM'):
    k = int(best.split('_')[1])
    model = GaussianMixture(n_components=k, covariance_type='full', random_state=42)
    final_df['cluster'] = model.fit_predict(X_reduced)
elif best.startswith('Spectral'):
    k = int(best.split('_')[1])
    model = SpectralClustering(n_clusters=k, affinity='nearest_neighbors', random_state=42)
    final_df['cluster'] = model.fit_predict(X_reduced)
elif best.startswith('Agglomerative'):
    k = int(best.split('_')[1])
    model = AgglomerativeClustering(n_clusters=k, metric='cosine', linkage='average')
    final_df['cluster'] = model.fit_predict(X_reduced)
else:  # DBSCAN
    final_df['cluster'] = lbl

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Reduced to 20 UMAP dimensions


/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ma

Spectral_2: Silhouette = 0.485
Agglomerative_2: Silhouette = 0.485
Agglomerative_9: Silhouette = 0.408
GMM_12: Silhouette = 0.407
GMM_8: Silhouette = 0.406
GMM_11: Silhouette = 0.401
Agglomerative_10: Silhouette = 0.399
Agglomerative_11: Silhouette = 0.399
GMM_2: Silhouette = 0.391
Agglomerative_5: Silhouette = 0.391
Agglomerative_12: Silhouette = 0.388
Agglomerative_8: Silhouette = 0.385
GMM_4: Silhouette = 0.383
GMM_9: Silhouette = 0.382
Spectral_6: Silhouette = 0.382
GMM_10: Silhouette = 0.382
GMM_6: Silhouette = 0.381
GMM_7: Silhouette = 0.380
Spectral_3: Silhouette = 0.376
Spectral_8: Silhouette = 0.370
Agglomerative_7: Silhouette = 0.367
Spectral_10: Silhouette = 0.357
Spectral_4: Silhouette = 0.354
Agglomerative_6: Silhouette = 0.341
GMM_5: Silhouette = 0.338
Spectral_7: Silhouette = 0.334
Spectral_5: Silhouette = 0.333
Agglomerative_4: Silhouette = 0.325
Spectral_12: Silhouette = 0.320
Spectral_11: Silhouette = 0.319
Agglomerative_3: Silhouette = 0.313
Spectral_9: Silhouette = 

/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [36]:
# Refit Spectral with best k=6
spec = SpectralClustering(n_clusters=6, affinity='nearest_neighbors', random_state=42)
final_df['cluster'] = spec.fit_predict(X_reduced)
df = pd.read_csv(os.path.join(path, "modern_teen_mental_health_main.csv"))
df = df.merge(final_df[['student_id', 'cluster']], on='student_id', how='left')

/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [37]:
final_df

,student_id,age,gender_Male,gender_Other,mood_friday,mood_monday,mood_saturday,mood_sunday,mood_thursday,mood_tuesday,...,meditated_today_True_tuesday,meditated_today_True_wednesday,used_ai_today_True_friday,used_ai_today_True_monday,used_ai_today_True_saturday,used_ai_today_True_sunday,used_ai_today_True_thursday,used_ai_today_True_tuesday,used_ai_today_True_wednesday,cluster
0,S0001,15,False,False,7.75,5.00,6.75,7.75,4.4,5.00,...,False,False,False,False,True,True,False,False,False,1
1,S0002,17,False,True,6.50,7.75,5.00,7.25,6.6,4.50,...,False,False,True,True,False,True,False,True,False,0
2,S0003,17,False,False,5.75,4.50,6.50,5.75,5.0,7.00,...,False,False,False,False,True,False,True,True,True,0
3,S0004,17,False,False,6.25,4.25,4.75,6.00,6.8,5.00,...,False,False,True,False,True,False,True,True,False,0
4,S0005,16,True,False,4.50,4.75,6.25,6.25,6.8,6.75,...,False,False,False,True,True,True,True,True,True,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,S0996,14,False,True,6.75,5.75,6.50,6.00,6.2,5.00,...,False,False,True,False,True,False,True,False,True,3
996,S0997,14,False,False,5.75,5.50,8.25,6.25,5.8,6.50,...,False,False,True,False,False,False,False,True,True,1
997,S0998,16,True,False,4.50,6.50,7.00,4.25,6.4,6.75,...,True,False,False,True,False,True,True,True,False,2
998,S0999,17,False,True,4.00,5.50,6.25,6.00,6.8,5.75,...,False,False,False,True,True,False,True,False,True,4


## Check for Non-Linear Correla

In [38]:
non_numeric_cols = ['student_id', 'date', 'day_name']
df_corr = final_df.drop(columns=non_numeric_cols, errors='ignore')
df_corr = df_corr.select_dtypes(include=[np.number, 'bool']).astype(float)

X = df_corr.drop(columns='cluster')
y = df_corr['cluster'].astype(int)

mi_scores = mutual_info_classif(X, y, discrete_features='auto')
mi_series = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)

chunk_size = 10
print("Features sorted by mutual information with 'cluster':\n")

for i in range(0, len(mi_series), chunk_size):
    chunk = mi_series.iloc[i:i+chunk_size]
    print(chunk.round(4).to_string())
    print("-" * 50)

Features sorted by mutual information with 'cluster':

meditated_today_True_saturday    0.2399
meditated_today_True_tuesday     0.2268
meditated_today_True_friday      0.1824
meditated_today_True_monday      0.1669
meditated_today_True_sunday      0.1288
support_feeling_tuesday          0.0457
mood_wednesday                   0.0345
journaled_today_True_monday      0.0315
mood_friday                      0.0258
screen_time_hours_tuesday        0.0256
--------------------------------------------------
stress_level_friday                   0.0230
sleep_hours_monday                    0.0203
journaled_today_True_thursday         0.0196
sleep_hours_tuesday                   0.0195
age                                   0.0185
support_feeling_saturday              0.0172
social_interaction_rating_saturday    0.0169
support_feeling_thursday              0.0151
stress_level_monday                   0.0134
sleep_hours_sunday                    0.0124
--------------------------------------------

### Interpretation of Mutual Information Scores with 'cluster'

#### 1. Strongest Differentiators (High MI)
- `meditated_today_True_saturday`: 0.2399  
- `meditated_today_True_tuesday`: 0.2268  
- `meditated_today_True_friday`: 0.1824  
- `meditated_today_True_monday`: 0.1669  
- `meditated_today_True_sunday`: 0.1288  

**Clusters differ most by meditation patterns**, especially on Saturday and Tuesday. Meditation is the most informative behavior.

---

#### 2. Moderate Contributors
- `support_feeling_tuesday`: 0.0457  
- `mood_wednesday`: 0.0345  
- `journaled_today_True_monday`: 0.0315  
- `mood_friday`: 0.0258  
- `screen_time_hours_tuesday`: 0.0256  

These features offer **some explanatory power**, likely reflecting emotional and digital behavior differences.

---

#### 3. Low or Irrelevant Features (MI ≈ 0)
- Most variables related to:
  - `support_feeling_*`  
  - `journaled_today_*`  
  - `used_ai_today_*`  
  - `screen_time_hours_*`  
  - `social_interaction_rating_*`  
- Also: `age`, `gender`, weekend mood/stress/sleep.

These features are **homogeneous across clusters** or **not aligned** with cluster distinctions.

---

### Summary
- **Meditation habits** are the primary drivers of clustering.
- **Mood and journaling** offer secondary signals.
- **Many variables** are redundant or irrelevant for distinguishing groups.

## Cluster Mean Profiles



In [39]:
# 1. Select top N features by MI
N = 15
top_mi_features = mi_series.nlargest(N).index.tolist()

# 2. Ensure they exist in your DataFrame
top_mi_features = [f for f in top_mi_features if f in final_df.columns]

# 3. Compute cluster profiles on MI‐selected features
profile_df = final_df.groupby('cluster')[top_mi_features].mean()
profile_df['cluster_size'] = final_df['cluster'].value_counts().sort_index()

# 7. Print the profiles
print("Cluster profiles (means of top MI features):")
display(profile_df)

Cluster profiles (means of top MI features):


,meditated_today_True_saturday,meditated_today_True_tuesday,meditated_today_True_friday,meditated_today_True_monday,meditated_today_True_sunday,support_feeling_tuesday,mood_wednesday,journaled_today_True_monday,mood_friday,screen_time_hours_tuesday,stress_level_friday,sleep_hours_monday,journaled_today_True_thursday,sleep_hours_tuesday,age,cluster_size
cluster,,,,,,,,,,,,,,,,
0,0.000000,0.002950,0.004425,0.028024,0.017699,6.572271,5.955457,0.351032,6.009218,6.956822,4.033186,6.465560,0.507375,6.539159,15.535398,678
1,1.000000,0.074627,0.059701,0.029851,0.104478,6.608209,5.898507,0.313433,6.179104,6.832463,3.876866,6.457463,0.522388,6.414179,15.358209,67
2,0.000000,0.942857,0.085714,0.128571,0.042857,6.557143,5.971429,0.271429,6.146429,7.013571,3.910714,6.443929,0.557143,6.410714,15.557143,70
3,0.014925,0.014925,0.910448,0.119403,0.044776,6.511194,5.823881,0.208955,6.044776,6.991791,3.951493,6.436194,0.507463,6.472761,15.626866,67
4,0.000000,0.000000,0.015873,0.984127,0.015873,6.539683,6.000000,0.333333,6.174603,7.012302,3.857143,6.492063,0.380952,6.396825,15.650794,63
5,0.000000,0.036364,0.018182,0.018182,0.945455,6.622727,6.123636,0.400000,5.877273,7.107273,4.204545,6.547273,0.454545,6.793636,15.872727,55


### 🔍 Key Patterns

### Cluster Profile Interpretation (Top MI Features)

#### Cluster 0 (n = 678)
- Very low meditation on any day.
- High screen time (6.96), moderate stress (4.03).
- Strong support feeling (6.57), average mood and journaling.
- Age ~15.5.

**Interpretation**:  
This is the **baseline group** — students with **low mindfulness**, **high digital engagement**, and **average emotional indicators**. Its large size reflects a **common, unengaged behavioral pattern**.

---

#### Cluster 1 (n = 67)
- 100% meditate on Saturday, some on Sunday.
- High support (6.61), solid mood (6.18), average journaling.
- Slightly lower stress and screen time.
- Age ~15.4.

**Interpretation**:  
**Weekend meditators** with **high support and mood**, emotionally balanced.

---

#### Cluster 2 (n = 70)
- Very high meditation on Tuesday (0.94).
- High support and mood, most journaling on Thursday (0.56).
- Balanced screen time and stress.
- Age ~15.6.

**Interpretation**:  
**Structured weekday meditators**, emotionally healthy and reflective.

---

#### Cluster 3 (n = 67)
- Very high Friday meditation (0.91).
- Good mood (6.04), lowest journaling (0.21).
- Balanced support and screen time.
- Age ~15.6.

**Interpretation**:  
**End-of-week meditators** with strong mood but **low reflection habits**.

---

#### Cluster 4 (n = 63)
- Very high Monday meditation (0.98).
- Strong mood (6.17), lowest stress (3.86).
- High screen time (7.01), low journaling on Thursday.
- Age ~15.7.

**Interpretation**:  
**Routine-focused meditators** with **low stress** and positive regulation.

---

#### Cluster 5 (n = 55)
- Very high Sunday meditation (0.95).
- Highest screen time (7.11) and stress (4.20).
- Mood slightly lower (5.88), moderate journaling.
- Oldest group (~15.87).

**Interpretation**:  
**Reflective Sunday meditators** who show **higher stress and screen time**, possibly end-of-week burnout.

---

### Summary
- Clusters are clearly shaped by **day-specific meditation behavior**.
- Mood, support, journaling, and screen time follow distinct patterns.
- **Cluster 0** represents the **default profile** of non-meditators — large but meaningful.